# 1. 라이브러리 및 패키지 불러오기

In [213]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#2. 데이터 불러오기

In [214]:
log_data = pd.read_csv('/content/drive/MyDrive/2022_Bigcontest/Data/log_data.csv')

In [215]:
relog = pd.read_csv('/content/drive/MyDrive/2022_Bigcontest/Data/relog.csv')

In [216]:
relog = relog.drop(['Unnamed: 0'], axis = 1)

# 3. 전처리

##1) mp_os
- android os는 getcreditinfo행동밖에 없고, Android os는 getcreditinfo만 없으므로 합쳐주는게 맞다고 판단 android -> Android로 변환

- 애플 제품에는 login이 없음 / 아이패드에는 여윳돈 계산 기능이 없는것인지 하지 않은 것인지 암튼 없음

In [217]:
log_data.loc[list(log_data[log_data['mp_os']=='android'].index),'mp_os'] = 'Android'

In [218]:
log_data.loc[list(log_data[log_data['mp_os']=='iPadOS'].index),'mp_os'] = 'iOS'

In [219]:
# mp_os가 결측값이면 mp_app_version도 결측값임을 확인

# 4. 파생변수 추가

## 1) 각 유저가 하루에 행동을 얼마나 했는가?

In [220]:
log = log_data

In [221]:
grouped = log.groupby(['user_id', 'date_cd'])['user_id'].count()
df_grouped = pd.DataFrame(grouped)

In [222]:
df_grouped.rename(columns = {'user_id' : 'user_id_cnt'}, inplace = True)

In [223]:
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new = df_grouped_new.reset_index('user_id')

In [224]:
df_grouped_new.head()

,user_id,date_cd,user_id_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12


## 2) 유저가 했던 행동을 총합해서 하나의 컬럼으로 추출

In [225]:
sample = pd.DataFrame(df_grouped_new['user_id_cnt'].groupby(df_grouped_new['user_id']).sum())

In [226]:
log = pd.merge(relog , sample,
                 how = 'left',
                 on = 'user_id')
log

,user_id,timestamp,event,user_id_cnt
0,1,2022-05-03 14:52:28,11-11-5-7-11,5
1,7,2022-05-22 16:39:49,11,1
2,9,2022-05-21 23:37:58,11-11-11,3
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120
...,...,...,...,...
584631,879693,2022-05-13 11:29:49,1-2-8-3-4-1-3-4-5-11-1-3-4-1-2-8-1-2-8-1-3-3-3...,61
584632,879694,2022-03-31 20:07:23,1-2-8-3-3-3,6
584633,879695,2022-05-27 12:48:32,1-2-8-3,4
584634,879696,2022-03-14 05:35:34,7-5-11-11-5-11-4-7-2-8-3,11


## 3) 유저가 기간동안 총 몇일 접속했는지

In [227]:
grouped = log_data.groupby(['user_id', 'date_cd'])['user_id'].count()
df_grouped = pd.DataFrame(grouped)

In [228]:
df_grouped.rename(columns = {'user_id' : 'user_id_day_cnt'}, inplace = True)
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new = df_grouped_new.reset_index('user_id')
df_grouped_new.head()

,user_id,date_cd,user_id_day_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12


In [229]:
df_grouped_new['user_id_day_cnt'].groupby(df_grouped_new['user_id']).count()

user_id
1          2
7          1
9          1
11         4
12        14
          ..
879693     8
879694     1
879695     1
879696     1
879698     1
Name: user_id_day_cnt, Length: 584636, dtype: int64

In [230]:
sample1=pd.DataFrame(df_grouped_new['user_id_day_cnt'].groupby(df_grouped_new['user_id']).count())
sample1.head()

,user_id_day_cnt
user_id,
1,2
7,1
9,1
11,4
12,14


In [231]:
log = pd.merge(log , sample1,
                 how = 'left',
                 on = 'user_id')
log.head()

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt
0,1,2022-05-03 14:52:28,11-11-5-7-11,5,2
1,7,2022-05-22 16:39:49,11,1,1
2,9,2022-05-21 23:37:58,11-11-11,3,1
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43,4
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120,14


## 4) 한도 조회, 신용 정보 조회, 대출 관리 서비스 이용, 여윳돈 계산기 서비스 이용, DSR 계산기 서비스 이용 중 어떤 것을 먼저 이용한 사람인가에 따른 차이점

In [232]:
log_1 = log_data[log_data['event'] == 'EndLoanApply']
log_2 = log_1.append(log_data[log_data['event'] == 'UseLoanManage'])
log_3 = log_2.append(log_data[log_data['event'] == 'UsePrepayCalc'])
log_4 = log_3.append(log_data[log_data['event'] == 'UseDSRCalc'])
log_new = log_4.append(log_data[log_data['event'] == 'GetCreditInfo'])

In [233]:
group = log_new.groupby(['user_id', 'event'])['timestamp'].min()
df_group = pd.DataFrame(group)

In [234]:
df_group_new = df_group.reset_index()
df_final = df_group_new.drop_duplicates(['user_id'], keep = 'first')
df_final.rename(columns = {'event':'first_event'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [235]:
sample3 = df_final[['user_id','first_event']]

In [236]:
log = pd.merge(log , sample3,
                 how = 'left',
                 on = 'user_id')
log

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event
0,1,2022-05-03 14:52:28,11-11-5-7-11,5,2,GetCreditInfo
1,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo
2,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43,4,EndLoanApply
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120,14,EndLoanApply
...,...,...,...,...,...,...
584631,879693,2022-05-13 11:29:49,1-2-8-3-4-1-3-4-5-11-1-3-4-1-2-8-1-2-8-1-3-3-3...,61,8,EndLoanApply
584632,879694,2022-03-31 20:07:23,1-2-8-3-3-3,6,1,EndLoanApply
584633,879695,2022-05-27 12:48:32,1-2-8-3,4,1,EndLoanApply
584634,879696,2022-03-14 05:35:34,7-5-11-11-5-11-4-7-2-8-3,11,1,EndLoanApply


# 5. Timeout 10분을 기준으로 세션 나누기

In [237]:
log2 = log_data
log2 = log2.sort_values(["user_id", "timestamp"], ascending= True)

In [238]:
log2['timestamp'] = pd.to_datetime(log2['timestamp'])

In [239]:
log2.head()

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
11709372,1,GetCreditInfo,2022-05-03 14:52:28,Android,464,2022-05-03
11709374,1,GetCreditInfo,2022-05-03 14:52:35,Android,464,2022-05-03
2451691,1,UseLoanManage,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
2451693,1,Login,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
7071607,1,GetCreditInfo,2022-06-16 23:58:42,Android,464,2022-06-16


In [240]:
log2 = log2.drop(['mp_os', 'mp_app_version', 'date_cd'], axis = 1)

In [241]:
log2['timeout'] = log2.timestamp - log2.timestamp.shift()

In [242]:
log2.head()

,user_id,event,timestamp,timeout
11709372,1,GetCreditInfo,2022-05-03 14:52:28,NaT
11709374,1,GetCreditInfo,2022-05-03 14:52:35,0 days 00:00:07
2451691,1,UseLoanManage,2022-06-16 23:58:41,44 days 09:06:06
2451693,1,Login,2022-06-16 23:58:41,0 days 00:00:00
7071607,1,GetCreditInfo,2022-06-16 23:58:42,0 days 00:00:01


In [243]:
pastid = 0
userid = 0

for i in range(len(log2)):
  userid = log2['user_id'].values[i]
  if userid != pastid:
    log2['timeout'].values[i] = 0
  if  log2['timeout'].values[i] > 10000000000:
    log2['timeout'].values[i] = 0
  pastid = userid

In [244]:
log2["timeout"].values[1]

numpy.timedelta64(7000000000,'ns')

In [245]:
log2.head(8)

,user_id,event,timestamp,timeout
11709372,1,GetCreditInfo,2022-05-03 14:52:28,0 days 00:00:00
11709374,1,GetCreditInfo,2022-05-03 14:52:35,0 days 00:00:07
2451691,1,UseLoanManage,2022-06-16 23:58:41,0 days 00:00:00
2451693,1,Login,2022-06-16 23:58:41,0 days 00:00:00
7071607,1,GetCreditInfo,2022-06-16 23:58:42,0 days 00:00:01
10428909,7,GetCreditInfo,2022-05-22 16:39:49,0 days 00:00:00
9627339,9,GetCreditInfo,2022-05-21 23:37:58,0 days 00:00:00
9627368,9,GetCreditInfo,2022-05-21 23:43:33,0 days 00:00:00


In [246]:
grouped = log2.groupby(['user_id'])['timeout'].sum()
df_grouped = pd.DataFrame(grouped)

In [247]:
df_grouped

,timeout
user_id,
1,0 days 00:00:08
7,0 days 00:00:00
9,0 days 00:00:00
11,0 days 00:00:30
12,0 days 00:03:20
...,...
879693,0 days 00:01:26
879694,0 days 00:00:00
879695,0 days 00:00:00


In [248]:
df_grouped.rename(columns = {'timeout' : 'timeout_sum'}, inplace = True)

In [249]:
log = pd.merge(log , df_grouped,
                 how = 'left',
                 on = 'user_id')
log.head()

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event,timeout_sum
0,1,2022-05-03 14:52:28,11-11-5-7-11,5,2,GetCreditInfo,0 days 00:00:08
1,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo,0 days 00:00:00
2,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo,0 days 00:00:00
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43,4,EndLoanApply,0 days 00:00:30
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120,14,EndLoanApply,0 days 00:03:20


In [249]:
log.to_csv('/content/drive/MyDrive/2022_Bigcontest/Data/log_Preprocessing_김혜현.csv')